In [ ]:
#imports
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, load_img, img_to_array
import numpy as np

from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
class Metrics(keras.callbacks.Callback):
    """
    Implementation of custom metrics: Precision, Recall, F-Measure and Confusion Matrix
    """
    
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        
        for i in range(len(self.validation_data)):
            x_val, y_val = self.validation_data.__getitem__(i)

        print(y_val.shape)
        print(y_val[1])
        print(y_val[2])

        y_predict = np.asarray(model.predict(self.validation_data, steps = 1))
        print(y_predict.shape)

        with tf.Session() as sess:
            lab = [i for i in range(0,75)]
            print(y_val.shape)

            y_val = np.argmax(y_val, axis=1)
            y_predict = np.argmax(y_predict, axis=1)
            print("y_val: ", y_val)
            print("y_predict:", y_predict)
            print("\nMetrics for Epoch")
            print("Confusion Matrix:\n",confusion_matrix(y_val,y_predict, labels=lab))
            print("Recall: ", recall_score(y_val,y_predict, average=None, labels=lab))
            print("Precision: ", precision_score(y_val,y_predict, average = None, labels=lab))
            print("F1_score: ", f1_score(y_val,y_predict, average =None, labels=lab))
            print("\n") 
            self._data.append({
                'val_recall': recall_score(y_val, y_predict, average = None, labels=lab),
                'val_precision': precision_score(y_val, y_predict, average = None, labels=lab),
                'val_f1_score': f1_score(y_val,y_predict, average = None, labels=lab),
            })
            return

    def get_data(self):
        return self._data
    
metrics = Metrics()

In [ ]:
def lr_scheduler(epoch,lr):
    
    """
    Learning rate scheduler decays the learning rate by factor of 0.1 every 10 epochs after 20 epochs
    """
    decay_rate = 0.1
    if epoch==20:
        return lr*decay_rate
    elif epoch%10==0 and epoch >20:
        return lr*decay_rate
    return lr

# To put in the model, put it inside callbacks
LRScheduler = keras.callbacks.LearningRateScheduler(lr_scheduler,verbose=1)

In [ ]:
#Data Generator for Data augmentation. Edit possible
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split = 0.0017)

In [ ]:
#Sanity check for augmentation

img = load_img('../datasets/large_sample/almond_desert/6350.jpg')  # this is a PIL image
img=img.resize((28,28))
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
print(x.shape)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in train_datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='almond_desert', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [ ]:
train_generator = train_datagen.flow_from_directory(
        '../datasets/large_sample',  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical',
        subset = 'training')  # since we use binary_crossentropy loss, we need binary labels



In [ ]:
val_generator = train_datagen.flow_from_directory(
        '../datasets/large_sample',  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 150x150
        batch_size=27,
        class_mode='categorical',
        subset = 'validation')  # since we use binary_crossentropy loss, we need binary labels

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

In [ ]:
#Naive Model

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(75))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
print(model.summary())

In [ ]:
model.fit_generator(train_generator,
                   steps_per_epoch = 1,
                   epochs = 1, validation_data = val_generator, callbacks=[metrics])

In [ ]:
img = load_img('../datasets/large_sample/almond_desert/6350.jpg')  # this is a PIL image
img=img.resize((28,28))
display(img)
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

print(x.shape)

In [ ]:

y = model.predict(val_generator, steps = 1)

In [ ]:
labels[np.argmax(y[1])]

In [ ]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

In [ ]:
#Data Generator for Data augmentation. Edit possible
val_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split = 0.3)

In [ ]:
small_train_generator = val_datagen.flow_from_directory(
        '../datasets/small_sample',  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical',
        subset = 'training')

In [ ]:
small_val_generator = val_datagen.flow_from_directory(
        '../datasets/small_sample',  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical',
        subset = 'validation')

In [ ]:
for layer in model.layers[:13]:
    layer.trainable=False


In [ ]:
for i,layer in enumerate(model.layers):
  print(i,layer.trainable)

In [ ]:
model.pop()
model.add(Dense(25))
model.add(Activation('sigmoid'))

In [ ]:

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit_generator(small_train_generator,
                   steps_per_epoch = 100,
                   epochs = 1, validation_data = small_val_generator)

In [ ]:
img = load_img('../datasets/small_sample/black_rice_cake/10585.jpg')  # this is a PIL image
img=img.resize((28,28))
display(img)
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

print(x.shape)

In [ ]:
labelsSmall = (small_train_generator.class_indices)
labelsSmall = dict((v,k) for k,v in labelsSmall.items())
print(labelsSmall)

In [ ]:
y = model.predict(x)

In [ ]:
labels[np.argmax(y)]